In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect("task3.db")
cursor = conn.cursor()

In [9]:
schema_sql = """
DROP TABLE IF EXISTS order_items;
DROP TABLE IF EXISTS orders;
DROP TABLE IF EXISTS products;
DROP TABLE IF EXISTS customers;
DROP TABLE IF EXISTS categories;

CREATE TABLE customers (
  customer_id INTEGER PRIMARY KEY,
  customer_name TEXT,
  email TEXT,
  signup_date DATE
);

CREATE TABLE categories (
  category_id INTEGER PRIMARY KEY,
  category_name TEXT
);

CREATE TABLE products (
  product_id INTEGER PRIMARY KEY,
  product_name TEXT,
  category_id INTEGER,
  price NUMERIC
);

CREATE TABLE orders (
  order_id INTEGER PRIMARY KEY,
  customer_id INTEGER,
  order_date DATE,
  order_status TEXT,
  shipping_fee NUMERIC,
  discount NUMERIC,
  FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);

CREATE TABLE order_items (
  order_item_id INTEGER PRIMARY KEY,
  order_id INTEGER,
  product_id INTEGER,
  quantity INTEGER,
  unit_price NUMERIC,
  FOREIGN KEY (order_id) REFERENCES orders(order_id),
  FOREIGN KEY (product_id) REFERENCES products(product_id)
);

INSERT INTO customers VALUES
(1,'Alice Kumar','alice@example.com','2018-02-10'),
(2,'Ravi Patel','ravi@example.com','2019-07-05'),
(3,'Sunita Rao','sunita@example.com','2020-11-23');

INSERT INTO categories VALUES
(1,'Electronics'),
(2,'Furniture'),
(3,'Office Supplies');

INSERT INTO products VALUES
(1,'Laptop A',1,800),
(2,'Office Chair',2,150),
(3,'Printer',1,200),
(4,'Desk',2,300),
(5,'Notebook Pack',3,15);

INSERT INTO orders VALUES
(1001,1,'2021-11-15','Completed',10,0),
(1002,2,'2021-12-01','Completed',15,5),
(1003,1,'2022-01-20','Returned',0,0),
(1004,3,'2022-02-10','Completed',12,2);

INSERT INTO order_items VALUES
(1,1001,1,1,800),
(2,1001,5,3,15),
(3,1002,2,2,150),
(4,1002,3,1,200),
(5,1003,4,1,300),
(6,1004,5,10,15);
"""

cursor.executescript(schema_sql)
conn.commit()

print("Tables created and sample data inserted successfully!")


Tables created and sample data inserted successfully!


In [10]:
def run_sql(query):
    return pd.read_sql_query(query, conn)

In [11]:
run_sql("SELECT * FROM customers;")

,customer_id,customer_name,email,signup_date
0,1,Alice Kumar,alice@example.com,2018-02-10
1,2,Ravi Patel,ravi@example.com,2019-07-05
2,3,Sunita Rao,sunita@example.com,2020-11-23


In [12]:
run_sql("""
SELECT c.category_name,
       SUM(oi.quantity * oi.unit_price) AS revenue
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
JOIN categories c ON p.category_id = c.category_id
GROUP BY c.category_name
ORDER BY revenue DESC;
""")


,category_name,revenue
0,Electronics,1000
1,Furniture,600
2,Office Supplies,195


In [13]:
run_sql("""
SELECT * FROM orders
WHERE order_status = 'Completed'
ORDER BY order_date DESC;
""")


,order_id,customer_id,order_date,order_status,shipping_fee,discount
0,1004,3,2022-02-10,Completed,12,2
1,1002,2,2021-12-01,Completed,15,5
2,1001,1,2021-11-15,Completed,10,0


In [14]:
run_sql("""
SELECT
    c.category_name,
    SUM(oi.quantity * oi.unit_price) AS revenue
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
JOIN categories c ON p.category_id = c.category_id
GROUP BY c.category_name
ORDER BY revenue DESC;
""")


,category_name,revenue
0,Electronics,1000
1,Furniture,600
2,Office Supplies,195


In [15]:
run_sql("""
SELECT
    strftime('%Y-%m', o.order_date) AS month,
    SUM(oi.quantity * oi.unit_price) AS revenue
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
GROUP BY month
ORDER BY month;
""")


,month,revenue
0,2021-11,845
1,2021-12,500
2,2022-01,300
3,2022-02,150


In [16]:
run_sql("""
SELECT
    c.customer_name,
    SUM(oi.quantity * oi.unit_price) AS total_spent
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
GROUP BY c.customer_name
ORDER BY total_spent DESC
LIMIT 5;
""")


,customer_name,total_spent
0,Alice Kumar,1145
1,Ravi Patel,500
2,Sunita Rao,150


In [17]:
run_sql("""
SELECT customer_id, customer_name
FROM customers
WHERE customer_id NOT IN (
    SELECT customer_id
    FROM orders
    WHERE order_status = 'Returned'
);
""")


,customer_id,customer_name
0,2,Ravi Patel
1,3,Sunita Rao


In [18]:
run_sql("""
SELECT
    (SUM(oi.quantity * oi.unit_price) * 1.0) /
    COUNT(DISTINCT o.customer_id) AS ARPU
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id;
""")


,ARPU
0,598.333333


In [19]:
cursor.executescript("""
CREATE VIEW IF NOT EXISTS order_revenue AS
SELECT
  o.order_id,
  o.customer_id,
  o.order_date,
  SUM(oi.quantity * oi.unit_price) AS items_total,
  o.shipping_fee,
  o.discount,
  (SUM(oi.quantity * oi.unit_price) + o.shipping_fee - o.discount) AS order_total
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
GROUP BY o.order_id;
""")

print("View created successfully!")


View created successfully!


In [20]:
cursor.executescript("""
CREATE INDEX IF NOT EXISTS idx_orders_customer ON orders(customer_id);
CREATE INDEX IF NOT EXISTS idx_products_category ON products(category_id);
""")

print("Indexes created successfully!")


Indexes created successfully!


In [21]:
run_sql("""
SELECT * FROM order_revenue
ORDER BY order_total DESC
LIMIT 5;
""")


,order_id,customer_id,order_date,items_total,shipping_fee,discount,order_total
0,1001,1,2021-11-15,845,10,0,855
1,1002,2,2021-12-01,500,15,5,510
2,1003,1,2022-01-20,300,0,0,300
3,1004,3,2022-02-10,150,12,2,160


In [22]:
run_sql("""
SELECT
    product_id,
    product_name,
    COALESCE(price, 0) AS price_without_null
FROM products;
""")


,product_id,product_name,price_without_null
0,1,Laptop A,800
1,2,Office Chair,150
2,3,Printer,200
3,4,Desk,300
4,5,Notebook Pack,15
